In [16]:
import pandas as pd

In [17]:
df_exam = pd.read_csv('examination_feature_cardio.csv')
df_subj = pd.read_csv('subjective_feature_cardio.csv')
df_obj = pd.read_csv('objective_feature_cardio.csv')

In [18]:
df1 = df_obj.merge(df_exam, left_on = 'id', right_on = 'id')

In [19]:
df = df1.merge(df_subj, left_on = 'id', right_on = 'id')

In [20]:
df = df.drop(columns = ['id'])
df

,age,gender,height,weight,cardiovascular_disease,ap_hi,ap_lo,cholesterol,glucose,smoke,alcohol,active
0,18393,Female,168,62,No,110,80,normal,normal,No,No,Yes
1,20228,Male,156,85,Yes,140,90,well above normal,normal,No,No,Yes
2,18857,Male,165,64,Yes,130,70,well above normal,normal,No,No,No
3,17623,Female,169,82,Yes,150,100,normal,normal,No,No,Yes
4,17474,Male,156,56,No,100,60,normal,normal,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...
69995,19240,Female,168,76,No,120,80,normal,normal,Yes,No,Yes
69996,22601,Male,158,126,Yes,140,90,above normal,above normal,No,No,Yes
69997,19066,Female,183,105,Yes,180,90,well above normal,normal,No,Yes,No
69998,22431,Male,163,72,Yes,135,80,normal,well above normal,No,No,No


In [21]:
df['age'] = df['age'] / 365
df['age'] = df['age'].map("{:.1f}".format)
df

,age,gender,height,weight,cardiovascular_disease,ap_hi,ap_lo,cholesterol,glucose,smoke,alcohol,active
0,50.4,Female,168,62,No,110,80,normal,normal,No,No,Yes
1,55.4,Male,156,85,Yes,140,90,well above normal,normal,No,No,Yes
2,51.7,Male,165,64,Yes,130,70,well above normal,normal,No,No,No
3,48.3,Female,169,82,Yes,150,100,normal,normal,No,No,Yes
4,47.9,Male,156,56,No,100,60,normal,normal,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...
69995,52.7,Female,168,76,No,120,80,normal,normal,Yes,No,Yes
69996,61.9,Male,158,126,Yes,140,90,above normal,above normal,No,No,Yes
69997,52.2,Female,183,105,Yes,180,90,well above normal,normal,No,Yes,No
69998,61.5,Male,163,72,Yes,135,80,normal,well above normal,No,No,No


In [25]:
# Removing  outliers in the 'weight' coulmn that less than 30 Kg
df.drop(df[df['weight'] < 30].index, inplace = True)
# Removing  outliers in the 'height' coulmn that are less than  100
df.drop(df[df['height'] < 100].index, inplace = True)
# Removing  outliers in the 'height' coulmn that are greater than  210
df.drop(df[df['height'] > 210].index, inplace = True)

# Remove outliers in the 'ap_hi' coulmn that are less than 60
df.drop(df[df['ap_hi'] < 60].index, inplace = True)
# Remove outliers in the 'ap_lo' coulmn that are above 300
df.drop(df[df['ap_hi'] > 300].index, inplace = True)

# Remove outliers in the 'ap_lo' coulmn that are below 10
df.drop(df[df['ap_lo'] < 10].index, inplace = True)
# # Remove outliers in the 'ap_lo' coulmn that are greater than 100
df.drop(df[df['ap_lo'] >= 100].index, inplace = True)

df

,age,gender,height,weight,cardiovascular_disease,ap_hi,ap_lo,cholesterol,glucose,smoke,alcohol,active
0,50.4,Female,168,62,No,110,80,normal,normal,No,No,Yes
1,55.4,Male,156,85,Yes,140,90,well above normal,normal,No,No,Yes
2,51.7,Male,165,64,Yes,130,70,well above normal,normal,No,No,No
4,47.9,Male,156,56,No,100,60,normal,normal,No,No,No
5,60.0,Male,151,67,No,120,80,above normal,above normal,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...
69995,52.7,Female,168,76,No,120,80,normal,normal,Yes,No,Yes
69996,61.9,Male,158,126,Yes,140,90,above normal,above normal,No,No,Yes
69997,52.2,Female,183,105,Yes,180,90,well above normal,normal,No,Yes,No
69998,61.5,Male,163,72,Yes,135,80,normal,well above normal,No,No,No


In [30]:
#create BMI datapoint
#BMI = (weight x weight)/(height)

df["BMI"] = (df['weight']*df['weight'])/ df['height']

df['BMI'] = df['BMI'].map("{:.0f}".format)
df

,age,gender,height,weight,cardiovascular_disease,ap_hi,ap_lo,cholesterol,glucose,smoke,alcohol,active,BMI
0,50.4,Female,168,62,No,110,80,normal,normal,No,No,Yes,23
1,55.4,Male,156,85,Yes,140,90,well above normal,normal,No,No,Yes,46
2,51.7,Male,165,64,Yes,130,70,well above normal,normal,No,No,No,25
4,47.9,Male,156,56,No,100,60,normal,normal,No,No,No,20
5,60.0,Male,151,67,No,120,80,above normal,above normal,No,No,No,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,52.7,Female,168,76,No,120,80,normal,normal,Yes,No,Yes,34
69996,61.9,Male,158,126,Yes,140,90,above normal,above normal,No,No,Yes,100
69997,52.2,Female,183,105,Yes,180,90,well above normal,normal,No,Yes,No,60
69998,61.5,Male,163,72,Yes,135,80,normal,well above normal,No,No,No,32
